In [1]:
# Aidan McEnaney
# February 15th, 2023
# Modified code from https://github.com/julialiu18/automated-glacier-terminus

In [2]:
# Imports everything we will need for the following sections of code

import numpy as np
import pandas as pd
import os
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
import time
from PIL import Image
from pyproj import Proj
from ttictoc import tic,toc
from Xsmurf_functions import *
#from wtmm2d_termiuspick import * # This makes jupyter mad

In [3]:
# Chooses the image to use for the rest of the sections

infoMage = Image.open('images_in/snail.jpg')
#infoMage = Image.open('images_in/2_I10-20080327_175510-R_CC.pgm')
#infoMage = Image.open('images_in/glacier.png')
 
# summarize some details about the image
print(infoMage.format)
print(infoMage.size)
print(infoMage.mode)

# asarray() class is used to convert
# PIL images into NumPy arrays
nuMage = np.asarray(infoMage)

# <class 'numpy.ndarray'>
print(type(nuMage))
 
#  shape
print(nuMage.shape)

[dx,dy,mm,m,a] = wtmm2d_v2(nuMage, 'gauss', 10)

FileNotFoundError: [Errno 2] No such file or directory: 'images_in/snail.jpg'

In [ ]:
# Visualize outputs from wtmm2d:
fig, axs = plt.subplots(2,3,figsize=(15,10))
axs[0,0].imshow(dx, aspect='equal', cmap = 'gray', interpolation='none'); axs[0,0].set_title('dx') # x gradient
axs[0,1].imshow(dy, aspect='equal', cmap = 'gray', interpolation='none'); axs[0,1].set_title('dy') # y gradient

axs[0,2].imshow(a, aspect='equal', cmap = 'gray', interpolation='none'); axs[0,2].set_title('a') # argument            
axs[1,0].imshow(mm, aspect='equal', cmap = 'gray', interpolation='none', vmin = np.min(mm), vmax = np.max(m)); 
axs[1,0].set_title('mm') # modulus maxima (interpolated)
axs[1,1].imshow(m, aspect='equal', cmap = 'gray', interpolation='none',vmin = np.min(mm), vmax = np.max(m));
axs[1,1].set_title('m') # modulus
axs[-1, -1].axis('off')

# Image.fromarray(mm).save('mm.tif')   
# Image.fromarray(m).save('m.tif')   

# plt.savefig('glacier_WTMM_test.png',dpi=200)
plt.show()

In [ ]:
"""p = np.zeros((6,6), dtype=np.int)
p[1:6, 2:5] = 1
b = np.zeros((600,600), dtype=np.int)
b[100:400, 100:500] = 100"""
c = np.zeros((360,360), dtype=np.int)
c[5:340, 5:360] = 100

#c = np.zeros((648,615), dtype=np.int)
#c[250:500, 200:500] = 100

#plt.imshow(p); plt.show()
#plt.imshow(b); plt.show()
plt.imshow(c); plt.show()



#plt.imshow(box_array); plt.show()  # Display mask

In [ ]:
# Mask
masked_a = emask(c, a)
masked_mm = emask(c, mm)
masked_m = emask(c, m)

# Visualize masked outputs from wtmm2d:
fig, axs = plt.subplots(1,3,figsize=(15,10))

axs[0].imshow(masked_a, aspect='equal', cmap = 'viridis', interpolation='none'); axs[0].set_title('a')              
axs[1].imshow(masked_mm, aspect='equal', cmap = 'gray', interpolation='none', vmin = np.min(masked_mm), vmax = np.max(masked_mm)); 
axs[1].set_title('mm')
axs[2].imshow(masked_m, aspect='equal', cmap = 'gray', interpolation='none',vmin = np.min(masked_m), vmax = np.max(masked_m));
axs[2].set_title('m')
# plt.savefig('glacier_WTMM_test.png',dpi=200)
plt.show()

In [ ]:
######################################################################################
cmm = wtmmchains(masked_mm,masked_a,1,10,3) # chain at a specified scale
######################################################################################

cmm[0].show() # Prints out stuff that helps sanity check

cmm_passed = []
# Filter chains based on size threshold
for j in range(0, len(cmm)):
    if cmm[j].size > 0: # adjust this condition to threshold
        cmm_passed.append(cmm[j])
        
plt.figure(figsize=(10,10))
plt.imshow(masked_mm,cmap='gray')
plt.xlim([0, mm.shape[1]])
plt.ylim([0, mm.shape[0]])
plt.gca().set_aspect('equal'); plt.gca().invert_yaxis()
    
for k in range(0, len(cmm_passed)):
    plt.plot(cmm_passed[k].ix, cmm_passed[k].iy, 's-', markersize=0.5)
plt.show()

In [ ]:
######################################################################################
size_thresh = 0.4 # Size percentile across all images
mod_thresh = 0.7 # Linemeanmod percentile across all images
arg_thresh = 0.1 # left-right argument fraction
metric = 0 # 0 = mass, 1 = scaledmass, 2 = size
######################################################################################
"""
# read in Greenland Polar Stereographic coordinates
PSy = np.array(pd.read_csv(basepath+'Box'+BOI+'/'+subfoldername+'img_idx_PSy_Box'+BOI+'.csv',
                          delimiter=' ',header=None))
# read in Greenland Polar Stereographic coordinates
PSx = np.array(pd.read_csv(basepath+'Box'+BOI+'/'+subfoldername+'img_idx_PSx_Box'+BOI+'.csv',
                          delimiter=' ',header=None))
"""
mass_or_size = []
passed_chains = []
for chain in cmm_passed:
    if chain.linemeanmod > mod_thresh: # only chains that pass the mod threshold
        if chain.size > size_thresh: # only chains that pass the size threshold
            [passedargs, argfrac] = filter_args(chain.args, np.pi/3) # identify the left & right-pointing args
            if argfrac > arg_thresh: # only chains that pass the orientation threshold
                if metric == 0:
                    mass_or_size.append(chain.mass) # evaluate by mass (length*gradient value)
                elif metric == 1:
                    mass_or_size.append(chain.scaled_mass) # evaluate by scaled mass (mass/2**scale)
                else:
                    mass_or_size.append(chain.size)
                #PSxs = PSx[np.array(chain.ix,dtype=int)]
                #PSys = PSy[np.array(chain.iy,dtype=int)]
                passed_chains.append(chain)
                
top_chains = pd.DataFrame(list(zip(mass_or_size, passed_chains)), columns=['metric_val','chainobj'])
top_chains = top_chains.sort_values(by=['metric_val'],ascending=False) # sort chains with highest metric at top
top_chains = top_chains[:5].reset_index(drop=True) # and grab the top 5
top_chains 

In [ ]:
# visualize top chains:
colors = pl.cm.viridis(np.linspace(0,1,5)) # generate colors using a colormap
plt.figure(figsize=(8,8))
plt.imshow(np.array(nuMage), aspect='equal', cmap = 'gray')
for k in range(0, len(top_chains)):
    # plot chains, darker = better picks
    plt.plot(top_chains.iloc[k][1].ix, top_chains.iloc[k][1].iy, 's-', color=colors[k],markersize=0.1)
#     break
            
plt.show()